In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from nba_api.stats.endpoints import leaguegamelog, playergamelogs, teamgamelogs
from time import sleep
import time
from filterpy.kalman import KalmanFilter
from typing import Dict, Tuple
from tqdm import tqdm

from autogluon.tabular import TabularPredictor, TabularDataset
from nba_api.stats.endpoints import scoreboardv2
from datetime import datetime


In [8]:
DB_NAME = 'nba_stats.db'

query = """ SELECT
    tbs.SEASON_YEAR,
    tbs.TEAM_ID,
    tbs.TEAM_ABBREVIATION,
    tbs.GAME_ID,
    tbs.GAME_DATE,
    tbs.MATCHUP,
    tbs.WL,
    tbs.MIN,
    tbs.PTS,
    tbs.FGM,
    tbs.FGA,
    tbs.FG3M,
    tbs.FG3A,
    tbs.FTM,
    tbs.FTA,
    tbs.OREB,
    tbs.DREB,
    tbs.REB,
    tbs.AST,
    tbs.TOV,
    tbs.STL,
    tbs.BLK,
    tbs.PLUS_MINUS,
    tas.OFF_RATING,
    tas.DEF_RATING,
    tas.NET_RATING,
    tas.PACE,
    tas.PIE,
    tss.PCT_PTS_2PT_MR,
    tss.PCT_AST_2PM,
    tss.PCT_UAST_2PM,
    tss.PCT_AST_3PM,
    tss.PCT_UAST_3PM,
    ths.defensiveBoxOuts as DEFENSIVE_BOX_OUTS,
    ths.contestedShots2pt as CONTESTED_SHOTS_2PT,
    ths.contestedShots3pt as CONTESTED_SHOTS_3PT,
    ths.DEFLECTIONS,
    tts.DIST,
    tts.TCHS,
    tts.PASS,
    tts.CFGM,
    tts.CFGA,
    tts.UFGM,
    tts.UFGA,
    tts.DFGM,
    tts.DFGA,
    dsbp.OPP_FGM_C,
    dsbp.OPP_FGA_C,
    dsbp.OPP_FGM_F,
    dsbp.OPP_FGA_F,
    dsbp.OPP_FGM_G,
    dsbp.OPP_FGA_G,
    dsbp.OPP_FG3M_C,
    dsbp.OPP_FG3A_C,
    dsbp.OPP_FG3M_F,
    dsbp.OPP_FG3A_F,
    dsbp.OPP_FG3M_G,
    dsbp.OPP_FG3A_G,
    dsbp.OPP_FTM_C,
    dsbp.OPP_FTM_F,
    dsbp.OPP_FTM_G,
    dsbp.OPP_FTA_C,
    dsbp.OPP_FTA_F,
    dsbp.OPP_FTA_G,
    tms.PTS_OFF_TOV,
    tms.PTS_2ND_CHANCE,
    tms.PTS_FB,
    tms.PTS_PAINT,
    tms.OPP_PTS_OFF_TOV,
    tms.OPP_PTS_2ND_CHANCE,
    tms.OPP_PTS_FB,
    tms.OPP_PTS_PAINT,
    tms.BLK as BLOCKS,
    tms.BLKA as BLOCKED_ATT,
    tms.PF,
    tms.PFD,
    tslb.[Restricted Area FGM] as FGM_RESTRICTED,
    tslb.[Restricted Area FGA] as FGA_RESTRICTED,
    tslb.[In The Paint (Non-RA) FGM] as FGM_PAINT_NON_RA,
    tslb.[In The Paint (Non-RA) FGA] as FGA_PAINT_NON_RA,
    tslb.[Mid-Range FGM] as FGM_MIDRANGE,
    tslb.[Mid-Range FGA] as FGA_MIDRANGE,
    tslb.[Corner 3 FGM] as FGM_CORNER3,
    tslb.[Corner 3 FGA] as FGA_CORNER3,
    tslb.[Above the Break 3 FGM] as FGM_ABOVE_BREAK3,
    tslb.[Above the Break 3 FGA] as FGA_ABOVE_BREAK3,
    CASE WHEN MATCHUP like '%@%' THEN 0 ELSE 1 END as IS_HOME
FROM team_basic_stats tbs
LEFT JOIN team_advanced_stats tas ON tbs.TEAM_ID = tas.TEAM_ID AND tbs.GAME_ID = tas.GAME_ID
LEFT JOIN team_scoring_stats tss ON tbs.TEAM_ID = tss.TEAM_ID AND tbs.GAME_ID = tss.GAME_ID
LEFT JOIN team_hustle_stats ths ON tbs.TEAM_ID = ths.teamId AND tbs.GAME_ID = ths.gameId
LEFT JOIN team_track_stats tts ON tbs.TEAM_ID = tts.TEAM_ID AND tbs.GAME_ID = tts.GAME_ID
LEFT JOIN defensive_stats_by_position dsbp ON tbs.TEAM_ID = dsbp.TEAM_ID AND tbs.GAME_DATE = dsbp.Date
LEFT JOIN team_miscellaneous_stats tms ON tbs.TEAM_ID = tms.TEAM_ID AND tbs.GAME_ID = tms.GAME_ID
LEFT JOIN team_shot_location_boxscores tslb ON tbs.TEAM_ID = tslb.TEAM_ID AND tbs.GAME_DATE = tslb.Date
WHERE tbs.SEASON_YEAR BETWEEN '2016-17' AND '2024-25'
ORDER BY tbs.GAME_DATE, tbs.TEAM_ID"""

conn = sqlite3.connect(DB_NAME)
df_team = pd.read_sql(query, conn)



In [58]:
conn.close()

In [10]:

def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)
    df = df.rename(columns={'SEASON_YEAR': 'SEASON'})

    abbr_mapping = {'NJN': 'BKN',
                    'CHH': 'CHA',
                    'VAN': 'MEM',
                    'NOH': 'NOP',
                    'NOK': 'NOP',
                    'SEA': 'OKC'}

    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    df = df.fillna(0)
    
    return df


df_team_clean = clean_team_data(df_team)



In [11]:
def prep_for_aggregation_team(df):
    """This function...
    1) Removes categories that are percentages,
    as we will be averaging them and do not want to average 
    percentages. 
    2) Converts shooting percentage stats into raw values"""

    df['FG2M'] = df['FGM'] - df['FG3M']
    df['FG2A'] = df['FGA'] - df['FG3A']
    df['PTS_2PT_MR'] = (df['PTS'] * df['PCT_PTS_2PT_MR']) #.astype('int8')
    df['AST_2PM'] = (df['FG2M'] * df['PCT_AST_2PM']) #.astype('int8')
    df['AST_3PM'] = (df['FG3M'] * df['PCT_AST_3PM']) #.astype('int8')
    df['UAST_2PM'] = (df['FG2M'] * df['PCT_UAST_2PM']) #.astype('int8')
    df['UAST_3PM'] = (df['FG3M'] * df['PCT_UAST_3PM']) #.astype('int8')

    df['OPP_FG2M_G'] = df['OPP_FGM_G'] - df['OPP_FG3M_G']
    df['OPP_FG2A_G'] = df['OPP_FGA_G'] - df['OPP_FG3A_G']

    df['OPP_FG2M_F'] = df['OPP_FGM_F'] - df['OPP_FG3M_F']
    df['OPP_FG2A_F'] = df['OPP_FGA_F'] - df['OPP_FG3A_F']
    
    df['OPP_FG2M_C'] = df['OPP_FGM_C'] - df['OPP_FG3M_C']
    df['OPP_FG2A_C'] = df['OPP_FGA_C'] - df['OPP_FG3A_C']

    df['POINT_DIFF'] = df['PLUS_MINUS']
    df['RECORD'] = df['WL']
    df['TEAM_SCORE'] = df['PTS']
    
    # percentage_columns = [x for x in df.columns if 'PCT' in x]
    drop_cols= ['OPP_FGM_G', 'OPP_FGA_G', 'OPP_FGM_F', 'OPP_FGA_F', 'OPP_FGM_C', 'OPP_FGA_C']
    #                       'MIN', 'PIE', 'PIE']
        
    df = df.drop(columns = drop_cols)
    
    ## Reorder Columns


    return df


df_team_clean2 = prep_for_aggregation_team(df_team_clean)



In [12]:


def create_matchups(df):
    """This function makes each row a matchup between 
    team and opp so that I can aggregate stats for a team's opponents"""
    keep_cols = ['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']
    stat_cols = [x for x in df.columns if x not in keep_cols]
    

    
    matchups = pd.merge(df, df, how='left', on=['GAME_ID'], suffixes=['', '_opp'])
    matchups = matchups.loc[matchups['TEAM_ID'] != matchups['TEAM_ID_opp']]

    matchups = matchups.drop(columns = ['SEASON_opp', 'TEAM_ID_opp', 'TEAM_ABBREVIATION_opp', 'GAME_DATE_opp',
                                         'MATCHUP_opp', 'TEAM_ID_opp', 'WL_opp']
                 )
    
    matchups
    
    return matchups


matchups = create_matchups(df_team_clean2)



In [13]:
team_optimal_spans = {'PTS': 29,
 'FGM': 33,
 'FGA': 32,
 'FG3M': 28,
 'FG3A': 19,
 'FTM': 37,
 'FTA': 37,
 'OREB': 33,
 'DREB': 37,
 'REB': 33,
 'AST': 30,
 'TOV': 39,
 'STL': 36,
 'BLK': 35,
 'PLUS_MINUS': 38,
 'OFF_RATING': 37,
 'DEF_RATING': 43,
 'NET_RATING': 39,
 'PACE': 17,
 'PIE': 37,
 'PCT_PTS_2PT_MR': 20,
 'PCT_AST_2PM': 31,
 'PCT_UAST_2PM': 31,
 'PCT_AST_3PM': 37,
 'PCT_UAST_3PM': 38,
 'DEFENSIVE_BOX_OUTS': 11,
 'CONTESTED_SHOTS_2PT': 24,
 'CONTESTED_SHOTS_3PT': 25,
 'deflections': 30,
 'DIST': 43,
 'TCHS': 31,
 'PASS': 22,
 'CFGM': 29,
 'CFGA': 26,
 'UFGM': 34,
 'UFGA': 32,
 'DFGM': 31,
 'DFGA': 29,
 'OPP_FG3M_C': 10,
 'OPP_FG3A_C': 7,
 'OPP_FG3M_F': 18,
 'OPP_FG3A_F': 11,
 'OPP_FG3M_G': 19,
 'OPP_FG3A_G': 12,
 'OPP_FTM_C': 12,
 'OPP_FTM_F': 20,
 'OPP_FTM_G': 21,
 'OPP_FTA_C': 11,
 'OPP_FTA_F': 18,
 'OPP_FTA_G': 19,
 'PTS_OFF_TOV': 45,
 'PTS_2ND_CHANCE': 37,
 'PTS_FB': 31,
 'PTS_PAINT': 27,
 'OPP_PTS_OFF_TOV': 47,
 'OPP_PTS_2ND_CHANCE': 47,
 'OPP_PTS_FB': 52,
 'OPP_PTS_PAINT': 32,
 'BLOCKS': 35,
 'BLOCKED_ATT': 40,
 'PF': 33,
 'PFD': 29,
 'FGM_RESTRICTED': 25,
 'FGA_RESTRICTED': 24,
 'FGM_PAINT_NON_RA': 35,
 'FGA_PAINT_NON_RA': 27,
 'FGM_MIDRANGE': 21,
 'FGA_MIDRANGE': 16,
 'FGM_CORNER3': 44,
 'FGA_CORNER3': 35,
 'FGM_ABOVE_BREAK3': 31,
 'FGA_ABOVE_BREAK3': 21,
 'FG2M': 26,
 'FG2A': 22,
 'PTS_2PT_MR': 21,
 'AST_2PM': 24,
 'AST_3PM': 31,
 'UAST_2PM': 31,
 'UAST_3PM': 36,
 'OPP_FG2M_G': 11,
 'OPP_FG2A_G': 9,
 'OPP_FG2M_F': 9,
 'OPP_FG2A_F': 8,
 'OPP_FG2M_C': 7,
 'OPP_FG2A_C': 6,
 'MIN_opp': 16,
 'PTS_opp': 32,
 'FGM_opp': 34,
 'FGA_opp': 28,
 'FG3M_opp': 36,
 'FG3A_opp': 28,
 'FTM_opp': 43,
 'FTA_opp': 41,
 'OREB_opp': 50,
 'DREB_opp': 39,
 'REB_opp': 39,
 'AST_opp': 40,
 'TOV_opp': 34,
 'STL_opp': 41,
 'BLK_opp': 40,
 'PLUS_MINUS_opp': 38,
 'OFF_RATING_opp': 43,
 'DEF_RATING_opp': 37,
 'NET_RATING_opp': 39,
 'PACE_opp': 17,
 'PIE_opp': 36,
 'PCT_PTS_2PT_MR_opp': 35,
 'PCT_AST_2PM_opp': 43,
 'PCT_UAST_2PM_opp': 43,
 'PCT_AST_3PM_opp': 44,
 'PCT_UAST_3PM_opp': 44,
 'DEFENSIVE_BOX_OUTS_opp': 10,
 'CONTESTED_SHOTS_2PT_opp': 22,
 'CONTESTED_SHOTS_3PT_opp': 21,
 'deflections_opp': 36,
 'DIST_opp': 48,
 'TCHS_opp': 41,
 'PASS_opp': 39,
 'CFGM_opp': 36,
 'CFGA_opp': 33,
 'UFGM_opp': 44,
 'UFGA_opp': 36,
 'DFGM_opp': 26,
 'DFGA_opp': 24,
 'OPP_FG3M_C_opp': 52,
 'OPP_FG3A_C_opp': 50,
 'OPP_FG3M_F_opp': 36,
 'OPP_FG3A_F_opp': 30,
 'OPP_FG3M_G_opp': 33,
 'OPP_FG3A_G_opp': 30,
 'OPP_FTM_C_opp': 44,
 'OPP_FTM_F_opp': 40,
 'OPP_FTM_G_opp': 46,
 'OPP_FTA_C_opp': 44,
 'OPP_FTA_F_opp': 40,
 'OPP_FTA_G_opp': 42,
 'PTS_OFF_TOV_opp': 47,
 'PTS_2ND_CHANCE_opp': 47,
 'PTS_FB_opp': 52,
 'PTS_PAINT_opp': 32,
 'OPP_PTS_OFF_TOV_opp': 45,
 'OPP_PTS_2ND_CHANCE_opp': 37,
 'OPP_PTS_FB_opp': 31,
 'OPP_PTS_PAINT_opp': 27,
 'BLOCKS_opp': 40,
 'BLOCKED_ATT_opp': 35,
 'PF_opp': 29,
 'PFD_opp': 33,
 'FGM_RESTRICTED_opp': 31,
 'FGA_RESTRICTED_opp': 30,
 'FGM_PAINT_NON_RA_opp': 52,
 'FGA_PAINT_NON_RA_opp': 51,
 'FGM_MIDRANGE_opp': 40,
 'FGA_MIDRANGE_opp': 30,
 'FGM_CORNER3_opp': 48,
 'FGA_CORNER3_opp': 36,
 'FGM_ABOVE_BREAK3_opp': 46,
 'FGA_ABOVE_BREAK3_opp': 35,
 'IS_HOME_opp': 52,
 'FG2M_opp': 31,
 'FG2A_opp': 26,
 'PTS_2PT_MR_opp': 40,
 'AST_2PM_opp': 37,
 'AST_3PM_opp': 38,
 'UAST_2PM_opp': 35,
 'UAST_3PM_opp': 52,
 'OPP_FG2M_G_opp': 40,
 'OPP_FG2A_G_opp': 34,
 'OPP_FG2M_F_opp': 34,
 'OPP_FG2A_F_opp': 33,
 'OPP_FG2M_C_opp': 52,
 'OPP_FG2A_C_opp': 49,
 'POINT_DIFF_opp': 38,
 'RECORD_opp': 6,
 'TEAM_SCORE_opp': 32}


def apply_optimal_spans(df: pd.DataFrame, optimal_spans: dict, grouping_col: str) -> pd.DataFrame:
    # Create a copy of the input DataFrame to avoid modifying the original
    df = df.copy()
    
    # Sort the DataFrame
    df = df.sort_values([grouping_col, 'GAME_DATE'])
    
    # Pre-calculate all EWA columns at once
    ewa_columns = {}
    
    for feature, span in optimal_spans.items():
        grouped = df.groupby(grouping_col)[feature]
        ewa_values = grouped.transform(
            lambda x: x.ewm(span=span, adjust=False).mean().shift(1)
        )
        ewa_columns[f'{feature}_EWA'] = ewa_values
    
    # Combine all new columns at once using concat
    result = pd.concat([df, pd.DataFrame(ewa_columns)], axis=1)
    
    return result


df_team_with_ewa = apply_optimal_spans(matchups, team_optimal_spans, 'TEAM_ID')


df_team_with_ewa

,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,PIE,PCT_PTS_2PT_MR,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,DEFENSIVE_BOX_OUTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,deflections,DIST,TCHS,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,OPP_FG3M_C,OPP_FG3A_C,OPP_FG3M_F,OPP_FG3A_F,OPP_FG3M_G,OPP_FG3A_G,OPP_FTM_C,OPP_FTM_F,OPP_FTM_G,OPP_FTA_C,OPP_FTA_F,OPP_FTA_G,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLOCKS,BLOCKED_ATT,PF,PFD,FGM_RESTRICTED,FGA_RESTRICTED,FGM_PAINT_NON_RA,FGA_PAINT_NON_RA,FGM_MIDRANGE,FGA_MIDRANGE,FGM_CORNER3,FGA_CORNER3,FGM_ABOVE_BREAK3,FGA_ABOVE_BREAK3,IS_HOME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,OPP_FG2M_G,OPP_FG2A_G,OPP_FG2M_F,OPP_FG2A_F,OPP_FG2M_C,OPP_FG2A_C,POINT_DIFF,RECORD,TEAM_SCORE,MIN_opp,PTS_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,REB_opp,AST_opp,TOV_opp,STL_opp,BLK_opp,PLUS_MINUS_opp,OFF_RATING_opp,DEF_RATING_opp,NET_RATING_opp,PACE_opp,PIE_opp,PCT_PTS_2PT_MR_opp,PCT_AST_2PM_opp,PCT_UAST_2PM_opp,PCT_AST_3PM_opp,PCT_UAST_3PM_opp,DEFENSIVE_BOX_OUTS_opp,CONTESTED_SHOTS_2PT_opp,CONTESTED_SHOTS_3PT_opp,deflections_opp,DIST_opp,TCHS_opp,PASS_opp,CFGM_opp,CFGA_opp,UFGM_opp,UFGA_opp,DFGM_opp,DFGA_opp,OPP_FG3M_C_opp,OPP_FG3A_C_opp,OPP_FG3M_F_opp,OPP_FG3A_F_opp,OPP_FG3M_G_opp,OPP_FG3A_G_opp,OPP_FTM_C_opp,OPP_FTM_F_opp,OPP_FTM_G_opp,OPP_FTA_C_opp,OPP_FTA_F_opp,OPP_FTA_G_opp,PTS_OFF_TOV_opp,PTS_2ND_CHANCE_opp,PTS_FB_opp,PTS_PAINT_opp,OPP_PTS_OFF_TOV_opp,OPP_PTS_2ND_CHANCE_opp,OPP_PTS_FB_opp,OPP_PTS_PAINT_opp,BLOCKS_opp,BLOCKED_ATT_opp,PF_opp,PFD_opp,FGM_RESTRICTED_opp,FGA_RESTRICTED_opp,FGM_PAINT_NON_RA_opp,FGA_PAINT_NON_RA_opp,FGM_MIDRANGE_opp,FGA_MIDRANGE_opp,FGM_CORNER3_opp,FGA_CORNER3_opp,FGM_ABOVE_BREAK3_opp,FGA_ABOVE_BREAK3_opp,IS_HOME_opp,FG2M_opp,FG2A_opp,PTS_2PT_MR_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp,OPP_FG2M_G_opp,OPP_FG2A_G_opp,OPP_FG2M_F_opp,OPP_FG2A_F_opp,OPP_FG2M_C_opp,OPP_FG2A_C_opp,POINT_DIFF_opp,RECORD_opp,TEAM_SCORE_opp,PTS_EWA,FGM_EWA,FGA_EWA,FG3M_EWA,FG3A_EWA,FTM_EWA,FTA_EWA,OREB_EWA,DREB_EWA,REB_EWA,AST_EWA,TOV_EWA,STL_EWA,BLK_EWA,PLUS_MINUS_EWA,OFF_RATING_EWA,DEF_RATING_EWA,NET_RATING_EWA,PACE_EWA,PIE_EWA,PCT_PTS_2PT_MR_EWA,PCT_AST_2PM_EWA,PCT_UAST_2PM_EWA,PCT_AST_3PM_EWA,PCT_UAST_3PM_EWA,DEFENSIVE_BOX_OUTS_EWA,CONTESTED_SHOTS_2PT_EWA,CONTESTED_SHOTS_3PT_EWA,deflections_EWA,DIST_EWA,TCHS_EWA,PASS_EWA,CFGM_EWA,CFGA_EWA,UFGM_EWA,UFGA_EWA,DFGM_EWA,DFGA_EWA,OPP_FG3M_C_EWA,OPP_FG3A_C_EWA,OPP_FG3M_F_EWA,OPP_FG3A_F_EWA,OPP_FG3M_G_EWA,OPP_FG3A_G_EWA,OPP_FTM_C_EWA,OPP_FTM_F_EWA,OPP_FTM_G_EWA,OPP_FTA_C_EWA,OPP_FTA_F_EWA,OPP_FTA_G_EWA,PTS_OFF_TOV_EWA,PTS_2ND_CHANCE_EWA,PTS_FB_EWA,PTS_PAINT_EWA,OPP_PTS_OFF_TOV_EWA,OPP_PTS_2ND_CHANCE_EWA,OPP_PTS_FB_EWA,OPP_PTS_PAINT_EWA,BLOCKS_EWA,BLOCKED_ATT_EWA,PF_EWA,PFD_EWA,FGM_RESTRICTED_EWA,FGA_RESTRICTED_EWA,FGM_PAINT_NON_RA_EWA,FGA_PAINT_NON_RA_EWA,FGM_MIDRANGE_EWA,FGA_MIDRANGE_EWA,FGM_CORNER3_EWA,FGA_CORNER3_EWA,FGM_ABOVE_BREAK3_EWA,FGA_ABOVE_BREAK3_EWA,FG2M_EWA,FG2A_EWA,PTS_2PT_MR_EWA,AST_2PM_EWA,AST_3PM_EWA,UAST_2PM_EWA,UAST_3PM_EWA,OPP_FG2M_G_EWA,OPP_FG2A_G_EWA,OPP_FG2M_F_EWA,OPP_FG2A_F_EWA,OPP_FG2M_C_EWA,OPP_FG2A_C_EWA,MIN_opp_EWA,PTS_opp_EWA,FGM_opp_EWA,FGA_opp_EWA,FG3M_opp_EWA,FG3A_opp_EWA,FTM_opp_EWA,FTA_opp_EWA,OREB_opp_EWA,DREB_opp_EWA,REB_opp_EWA,AST_opp_EWA,TOV_opp_EWA,STL_opp_EWA,BLK_opp_EWA,PLUS_MINUS_opp_EWA,OFF_RATING_opp_EWA,DEF_RATING_opp_EWA,NET_RATING_opp_EWA,PACE_opp_EWA,PIE_opp_EWA,PCT_PTS_2PT_MR_opp_EWA,PCT_AST_2PM_opp_EWA,PCT_UAST_2PM_opp_EWA,PCT_AST_3PM_opp_EWA,PCT_UAST_3PM_opp_EWA,DEFENSIVE_BOX_OUTS_opp_EWA,CONTESTED_SHOTS_2PT_opp_EWA,CONTESTED_SHOTS_3PT_opp_EWA,deflections_opp_EWA,DIST_opp_EWA,TCHS_opp_EWA,PASS_opp_EWA,CFGM_opp_EWA,CFGA_opp_EWA,UFGM_opp_EWA,UFGA_opp_EWA,DFGM_opp_EWA,DFGA_opp_EWA,OPP_FG3M_C_opp_EWA,OPP_FG3A_C_opp_EWA,OPP_FG3M_F_opp_EWA,OPP_FG3A_F_opp_EWA,OPP_FG3M_G_o

In [54]:
import json
team_optimal_spans = {'PTS': 29,
 'FGM': 33,
 'FGA': 32,
 'FG3M': 28,
 'FG3A': 19,
 'FTM': 37,
 'FTA': 37,
 'OREB': 33,
 'DREB': 37,
 'REB': 33,
 'AST': 30,
 'TOV': 39,
 'STL': 36,
 'BLK': 35,
 'PLUS_MINUS': 38,
 'OFF_RATING': 37,
 'DEF_RATING': 43,
 'NET_RATING': 39,
 'PACE': 17,
 'PIE': 37,
 'PCT_PTS_2PT_MR': 20,
 'PCT_AST_2PM': 31,
 'PCT_UAST_2PM': 31,
 'PCT_AST_3PM': 37,
 'PCT_UAST_3PM': 38,
 'DEFENSIVE_BOX_OUTS': 11,
 'CONTESTED_SHOTS_2PT': 24,
 'CONTESTED_SHOTS_3PT': 25,
 'deflections': 30,
 'DIST': 43,
 'TCHS': 31,
 'PASS': 22,
 'CFGM': 29,
 'CFGA': 26,
 'UFGM': 34,
 'UFGA': 32,
 'DFGM': 31,
 'DFGA': 29,
 'OPP_FG3M_C': 10,
 'OPP_FG3A_C': 7,
 'OPP_FG3M_F': 18,
 'OPP_FG3A_F': 11,
 'OPP_FG3M_G': 19,
 'OPP_FG3A_G': 12,
 'OPP_FTM_C': 12,
 'OPP_FTM_F': 20,
 'OPP_FTM_G': 21,
 'OPP_FTA_C': 11,
 'OPP_FTA_F': 18,
 'OPP_FTA_G': 19,
 'PTS_OFF_TOV': 45,
 'PTS_2ND_CHANCE': 37,
 'PTS_FB': 31,
 'PTS_PAINT': 27,
 'OPP_PTS_OFF_TOV': 47,
 'OPP_PTS_2ND_CHANCE': 47,
 'OPP_PTS_FB': 52,
 'OPP_PTS_PAINT': 32,
 'BLOCKS': 35,
 'BLOCKED_ATT': 40,
 'PF': 33,
 'PFD': 29,
 'FGM_RESTRICTED': 25,
 'FGA_RESTRICTED': 24,
 'FGM_PAINT_NON_RA': 35,
 'FGA_PAINT_NON_RA': 27,
 'FGM_MIDRANGE': 21,
 'FGA_MIDRANGE': 16,
 'FGM_CORNER3': 44,
 'FGA_CORNER3': 35,
 'FGM_ABOVE_BREAK3': 31,
 'FGA_ABOVE_BREAK3': 21,
 'FG2M': 26,
 'FG2A': 22,
 'PTS_2PT_MR': 21,
 'AST_2PM': 24,
 'AST_3PM': 31,
 'UAST_2PM': 31,
 'UAST_3PM': 36,
 'OPP_FG2M_G': 11,
 'OPP_FG2A_G': 9,
 'OPP_FG2M_F': 9,
 'OPP_FG2A_F': 8,
 'OPP_FG2M_C': 7,
 'OPP_FG2A_C': 6,
 'PTS_opp': 32,
 'FGM_opp': 34,
 'FGA_opp': 28,
 'FG3M_opp': 36,
 'FG3A_opp': 28,
 'FTM_opp': 43,
 'FTA_opp': 41,
 'OREB_opp': 50,
 'DREB_opp': 39,
 'REB_opp': 39,
 'AST_opp': 40,
 'TOV_opp': 34,
 'STL_opp': 41,
 'BLK_opp': 40,
 'PLUS_MINUS_opp': 38,
 'OFF_RATING_opp': 43,
 'DEF_RATING_opp': 37,
 'NET_RATING_opp': 39,
 'PACE_opp': 17,
 'PIE_opp': 36,
 'PCT_PTS_2PT_MR_opp': 35,
 'PCT_AST_2PM_opp': 43,
 'PCT_UAST_2PM_opp': 43,
 'PCT_AST_3PM_opp': 44,
 'PCT_UAST_3PM_opp': 44,
 'DEFENSIVE_BOX_OUTS_opp': 10,
 'CONTESTED_SHOTS_2PT_opp': 22,
 'CONTESTED_SHOTS_3PT_opp': 21,
 'deflections_opp': 36,
 'DIST_opp': 48,
 'TCHS_opp': 41,
 'PASS_opp': 39,
 'CFGM_opp': 36,
 'CFGA_opp': 33,
 'UFGM_opp': 44,
 'UFGA_opp': 36,
 'DFGM_opp': 26,
 'DFGA_opp': 24,
 'OPP_FG3M_C_opp': 52,
 'OPP_FG3A_C_opp': 50,
 'OPP_FG3M_F_opp': 36,
 'OPP_FG3A_F_opp': 30,
 'OPP_FG3M_G_opp': 33,
 'OPP_FG3A_G_opp': 30,
 'OPP_FTM_C_opp': 44,
 'OPP_FTM_F_opp': 40,
 'OPP_FTM_G_opp': 46,
 'OPP_FTA_C_opp': 44,
 'OPP_FTA_F_opp': 40,
 'OPP_FTA_G_opp': 42,
 'PTS_OFF_TOV_opp': 47,
 'PTS_2ND_CHANCE_opp': 47,
 'PTS_FB_opp': 52,
 'PTS_PAINT_opp': 32,
 'OPP_PTS_OFF_TOV_opp': 45,
 'OPP_PTS_2ND_CHANCE_opp': 37,
 'OPP_PTS_FB_opp': 31,
 'OPP_PTS_PAINT_opp': 27,
 'BLOCKS_opp': 40,
 'BLOCKED_ATT_opp': 35,
 'PF_opp': 29,
 'PFD_opp': 33,
 'FGM_RESTRICTED_opp': 31,
 'FGA_RESTRICTED_opp': 30,
 'FGM_PAINT_NON_RA_opp': 52,
 'FGA_PAINT_NON_RA_opp': 51,
 'FGM_MIDRANGE_opp': 40,
 'FGA_MIDRANGE_opp': 30,
 'FGM_CORNER3_opp': 48,
 'FGA_CORNER3_opp': 36,
 'FGM_ABOVE_BREAK3_opp': 46,
 'FGA_ABOVE_BREAK3_opp': 35,
 'FG2M_opp': 31,
 'FG2A_opp': 26,
 'PTS_2PT_MR_opp': 40,
 'AST_2PM_opp': 37,
 'AST_3PM_opp': 38,
 'UAST_2PM_opp': 35,
 'UAST_3PM_opp': 52,
 'OPP_FG2M_G_opp': 40,
 'OPP_FG2A_G_opp': 34,
 'OPP_FG2M_F_opp': 34,
 'OPP_FG2A_F_opp': 33,
 'OPP_FG2M_C_opp': 52,
 'OPP_FG2A_C_opp': 49}

with open("C:/Users/Jordan Nishimura/nba_model_using_player_embeddings/config/optimal_spans.json", "w") as file:
    json.dump(team_optimal_spans, file, indent=4)


In [14]:

def add_percentage_features(df):
    """Add the following features for both team and opp:
    OREB_PCT, DREB_PCT, REB_PCT, TS_PCT, EFG_PCT, AST_RATIO, TOV_PCT, PIE.
    """
    
    df = df.copy()
    
    df['FG2_PCT'] = df['FG2M'] / df[f'FG2A']
    df['FG3_PCT'] = df['FG3M'] / df[f'FG3A']
    
    df['OREB_PCT'] = df['OREB'] / (df['OREB'] + df['DREB_opp'])
    df['OREB_PCT_opp'] = df['OREB_opp'] / (df['OREB_opp'] + df['DREB'])

    df['DREB_PCT'] = df['DREB'] / (df['DREB'] + df['OREB_opp'])
    df['DREB_PCT_opp'] = df['DREB_opp'] / (df['DREB_opp'] + df['OREB'])

    df['REB_PCT'] = df['REB'] / (df['REB'] + df['REB_opp'])
    df['REB_PCT_opp'] = df['REB_opp'] / (df['REB_opp'] + df['REB'])

    df['TS_PCT'] = df['PTS'] / ((2*(df['FG2A'] + df['FG3A']) + 0.44*df['FTA']))
    
    df['TS_PCT_opp'] = df['PTS_opp'] / ((2*(df['FG2A_opp'] + df['FG3A_opp']) + 0.44*df['FTA_opp']))

    df['EFG_PCT'] = (df['FG2M'] + 1.5*df['FG3M']) / (df['FG2A']
                                                                    + df['FG3A'])
    df['EFG_PCT_opp'] = (df['FG2M_opp'] + 1.5*df['FG3M_opp']) / (df['FG2A_opp'] 
                                                                 + df['FG3A_opp'])

    df['AST_RATIO'] = (df['AST'] * 100) / df['PACE']
    df['AST_RATIO_opp'] = (df['AST_opp'] * 100) / df['PACE_opp']

    df['TOV_PCT'] = 100*df['TOV'] / (df['FG2A'] 
                                               + df['FG3A'] 
                                               + 0.44*df['FTA'] 
                                               + df['TOV'])
    
    df['TOV_PCT_opp'] = 100*df['TOV_opp'] / (df['FG2A_opp'] 
                                             + df['FG3A_opp'] 
                                             + 0.44*df['FTA_opp'] 
                                             + df['TOV_opp'])
    
    
    df['PIE'] = ((df['PTS'] + df['FG2M'] + df['FG3M'] + df['FTM'] 
                 - df['FG2A'] - df['FG3A'] - df['FTA'] 
                 + df['DREB'] + df['OREB']/2
                + df['AST'] + df['STL'] + df['BLK']/2
                - df['PF'] - df['TOV']) 
                 / (df['PTS'] + df['PTS_opp'] + df['FG2M'] + df['FG2M_opp']
                   + df['FG3M'] + df['FG3M_opp'] + df['FTM'] + df['FTM_opp']
                   - df['FG2A'] - df['FG2A_opp'] - df['FG3A'] - df['FG3A_opp'] 
                    - df['FTA'] - df['FTA_opp'] + df['DREB'] + df['DREB_opp']
                    + (df['OREB']+df['OREB_opp'])/2 + df['AST'] + df['AST_opp']
                    + df['STL'] + df['STL_opp'] + (df['BLK'] + df['BLK_opp'])/2
                    - df['PF'] - df['PF_opp'] - df['TOV'] - df['TOV_opp']))
        
    return df
  
  
  
df_team_with_ewa2 = add_percentage_features(df_team_with_ewa)

df_team_with_ewa2


,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,PIE,PCT_PTS_2PT_MR,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,DEFENSIVE_BOX_OUTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,deflections,DIST,TCHS,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,OPP_FG3M_C,OPP_FG3A_C,OPP_FG3M_F,OPP_FG3A_F,OPP_FG3M_G,OPP_FG3A_G,OPP_FTM_C,OPP_FTM_F,OPP_FTM_G,OPP_FTA_C,OPP_FTA_F,OPP_FTA_G,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLOCKS,BLOCKED_ATT,PF,PFD,FGM_RESTRICTED,FGA_RESTRICTED,FGM_PAINT_NON_RA,FGA_PAINT_NON_RA,FGM_MIDRANGE,FGA_MIDRANGE,FGM_CORNER3,FGA_CORNER3,FGM_ABOVE_BREAK3,FGA_ABOVE_BREAK3,IS_HOME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,OPP_FG2M_G,OPP_FG2A_G,OPP_FG2M_F,OPP_FG2A_F,OPP_FG2M_C,OPP_FG2A_C,POINT_DIFF,RECORD,TEAM_SCORE,MIN_opp,PTS_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,REB_opp,AST_opp,TOV_opp,STL_opp,BLK_opp,PLUS_MINUS_opp,OFF_RATING_opp,DEF_RATING_opp,NET_RATING_opp,PACE_opp,PIE_opp,PCT_PTS_2PT_MR_opp,PCT_AST_2PM_opp,PCT_UAST_2PM_opp,PCT_AST_3PM_opp,PCT_UAST_3PM_opp,DEFENSIVE_BOX_OUTS_opp,CONTESTED_SHOTS_2PT_opp,CONTESTED_SHOTS_3PT_opp,deflections_opp,DIST_opp,TCHS_opp,PASS_opp,CFGM_opp,CFGA_opp,UFGM_opp,UFGA_opp,DFGM_opp,DFGA_opp,OPP_FG3M_C_opp,OPP_FG3A_C_opp,OPP_FG3M_F_opp,OPP_FG3A_F_opp,OPP_FG3M_G_opp,OPP_FG3A_G_opp,OPP_FTM_C_opp,OPP_FTM_F_opp,OPP_FTM_G_opp,OPP_FTA_C_opp,OPP_FTA_F_opp,OPP_FTA_G_opp,PTS_OFF_TOV_opp,PTS_2ND_CHANCE_opp,PTS_FB_opp,PTS_PAINT_opp,OPP_PTS_OFF_TOV_opp,OPP_PTS_2ND_CHANCE_opp,OPP_PTS_FB_opp,OPP_PTS_PAINT_opp,BLOCKS_opp,BLOCKED_ATT_opp,PF_opp,PFD_opp,FGM_RESTRICTED_opp,FGA_RESTRICTED_opp,FGM_PAINT_NON_RA_opp,FGA_PAINT_NON_RA_opp,FGM_MIDRANGE_opp,FGA_MIDRANGE_opp,FGM_CORNER3_opp,FGA_CORNER3_opp,FGM_ABOVE_BREAK3_opp,FGA_ABOVE_BREAK3_opp,IS_HOME_opp,FG2M_opp,FG2A_opp,PTS_2PT_MR_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp,OPP_FG2M_G_opp,OPP_FG2A_G_opp,OPP_FG2M_F_opp,OPP_FG2A_F_opp,OPP_FG2M_C_opp,OPP_FG2A_C_opp,POINT_DIFF_opp,RECORD_opp,TEAM_SCORE_opp,PTS_EWA,FGM_EWA,FGA_EWA,FG3M_EWA,FG3A_EWA,FTM_EWA,FTA_EWA,OREB_EWA,DREB_EWA,REB_EWA,AST_EWA,TOV_EWA,STL_EWA,BLK_EWA,PLUS_MINUS_EWA,OFF_RATING_EWA,DEF_RATING_EWA,NET_RATING_EWA,PACE_EWA,PIE_EWA,PCT_PTS_2PT_MR_EWA,PCT_AST_2PM_EWA,PCT_UAST_2PM_EWA,PCT_AST_3PM_EWA,PCT_UAST_3PM_EWA,DEFENSIVE_BOX_OUTS_EWA,CONTESTED_SHOTS_2PT_EWA,CONTESTED_SHOTS_3PT_EWA,deflections_EWA,DIST_EWA,TCHS_EWA,PASS_EWA,CFGM_EWA,CFGA_EWA,UFGM_EWA,UFGA_EWA,DFGM_EWA,DFGA_EWA,OPP_FG3M_C_EWA,OPP_FG3A_C_EWA,OPP_FG3M_F_EWA,OPP_FG3A_F_EWA,OPP_FG3M_G_EWA,OPP_FG3A_G_EWA,OPP_FTM_C_EWA,OPP_FTM_F_EWA,OPP_FTM_G_EWA,OPP_FTA_C_EWA,OPP_FTA_F_EWA,OPP_FTA_G_EWA,PTS_OFF_TOV_EWA,PTS_2ND_CHANCE_EWA,PTS_FB_EWA,PTS_PAINT_EWA,OPP_PTS_OFF_TOV_EWA,OPP_PTS_2ND_CHANCE_EWA,OPP_PTS_FB_EWA,OPP_PTS_PAINT_EWA,BLOCKS_EWA,BLOCKED_ATT_EWA,PF_EWA,PFD_EWA,FGM_RESTRICTED_EWA,FGA_RESTRICTED_EWA,FGM_PAINT_NON_RA_EWA,FGA_PAINT_NON_RA_EWA,FGM_MIDRANGE_EWA,FGA_MIDRANGE_EWA,FGM_CORNER3_EWA,FGA_CORNER3_EWA,FGM_ABOVE_BREAK3_EWA,FGA_ABOVE_BREAK3_EWA,FG2M_EWA,FG2A_EWA,PTS_2PT_MR_EWA,AST_2PM_EWA,AST_3PM_EWA,UAST_2PM_EWA,UAST_3PM_EWA,OPP_FG2M_G_EWA,OPP_FG2A_G_EWA,OPP_FG2M_F_EWA,OPP_FG2A_F_EWA,OPP_FG2M_C_EWA,OPP_FG2A_C_EWA,MIN_opp_EWA,PTS_opp_EWA,FGM_opp_EWA,FGA_opp_EWA,FG3M_opp_EWA,FG3A_opp_EWA,FTM_opp_EWA,FTA_opp_EWA,OREB_opp_EWA,DREB_opp_EWA,REB_opp_EWA,AST_opp_EWA,TOV_opp_EWA,STL_opp_EWA,BLK_opp_EWA,PLUS_MINUS_opp_EWA,OFF_RATING_opp_EWA,DEF_RATING_opp_EWA,NET_RATING_opp_EWA,PACE_opp_EWA,PIE_opp_EWA,PCT_PTS_2PT_MR_opp_EWA,PCT_AST_2PM_opp_EWA,PCT_UAST_2PM_opp_EWA,PCT_AST_3PM_opp_EWA,PCT_UAST_3PM_opp_EWA,DEFENSIVE_BOX_OUTS_opp_EWA,CONTESTED_SHOTS_2PT_opp_EWA,CONTESTED_SHOTS_3PT_opp_EWA,deflections_opp_EWA,DIST_opp_EWA,TCHS_opp_EWA,PASS_opp_EWA,CFGM_opp_EWA,CFGA_opp_EWA,UFGM_opp_EWA,UFGA_opp_EWA,DFGM_opp_EWA,DFGA_opp_EWA,OPP_FG3M_C_opp_EWA,OPP_FG3A_C_opp_EWA,OPP_FG3M_F_opp_EWA,OPP_FG3A_F_opp_EWA,OPP_FG3M_G_o

In [20]:

# def create_matchup_df2(df):
#     df_team_merged = pd.merge(df, df, on=['GAME_ID', 'GAME_DATE'], suffixes=('_tm', '_opp'))
#     df_team_merged = df_team_merged.loc[df_team_merged['TEAM_ID_tm'] != df_team_merged['TEAM_ID_opp']]
#     df_team_merged = df_team_merged.sort_values(['GAME_DATE', 'GAME_ID', 'IS_HOME_tm'])

#     keep_cols = ['SEASON_tm', 'TEAM_ID_tm', 'TEAM_ABBREVIATION_tm', 'TEAM_ID_opp', 'TEAM_ABBREVIATION_opp', 'GAME_ID', 'GAME_DATE', 'MATCHUP_tm', 'WL_tm', 'PTS_tm', 'PTS_opp']
#     ewa_cols = [col for col in df_team_merged.columns if "EWA" in col]
#     keep_cols.extend(ewa_cols)
    
#     df_team_merged = df_team_merged.rename(columns={'EWA_tm': 'EWA_tm_tm', 'EWA_opp': 'EWA_tm_opp'})
    
#     return df_team_merged[keep_cols] 


# df_matchup_team = create_matchup_df2(df_team_with_ewa2)


In [15]:
keep_cols = ['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'PTS']
ewa_cols = [col for col in df_team_with_ewa2.columns if "EWA" in col]
keep_cols.extend(ewa_cols)

df_team_with_ewa3 = df_team_with_ewa2[keep_cols]

In [16]:
df_team_with_ewa3

,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PTS_EWA,FGM_EWA,FGA_EWA,FG3M_EWA,FG3A_EWA,FTM_EWA,FTA_EWA,OREB_EWA,DREB_EWA,REB_EWA,AST_EWA,TOV_EWA,STL_EWA,BLK_EWA,PLUS_MINUS_EWA,OFF_RATING_EWA,DEF_RATING_EWA,NET_RATING_EWA,PACE_EWA,PIE_EWA,PCT_PTS_2PT_MR_EWA,PCT_AST_2PM_EWA,PCT_UAST_2PM_EWA,PCT_AST_3PM_EWA,PCT_UAST_3PM_EWA,DEFENSIVE_BOX_OUTS_EWA,CONTESTED_SHOTS_2PT_EWA,CONTESTED_SHOTS_3PT_EWA,deflections_EWA,DIST_EWA,TCHS_EWA,PASS_EWA,CFGM_EWA,CFGA_EWA,UFGM_EWA,UFGA_EWA,DFGM_EWA,DFGA_EWA,OPP_FG3M_C_EWA,OPP_FG3A_C_EWA,OPP_FG3M_F_EWA,OPP_FG3A_F_EWA,OPP_FG3M_G_EWA,OPP_FG3A_G_EWA,OPP_FTM_C_EWA,OPP_FTM_F_EWA,OPP_FTM_G_EWA,OPP_FTA_C_EWA,OPP_FTA_F_EWA,OPP_FTA_G_EWA,PTS_OFF_TOV_EWA,PTS_2ND_CHANCE_EWA,PTS_FB_EWA,PTS_PAINT_EWA,OPP_PTS_OFF_TOV_EWA,OPP_PTS_2ND_CHANCE_EWA,OPP_PTS_FB_EWA,OPP_PTS_PAINT_EWA,BLOCKS_EWA,BLOCKED_ATT_EWA,PF_EWA,PFD_EWA,FGM_RESTRICTED_EWA,FGA_RESTRICTED_EWA,FGM_PAINT_NON_RA_EWA,FGA_PAINT_NON_RA_EWA,FGM_MIDRANGE_EWA,FGA_MIDRANGE_EWA,FGM_CORNER3_EWA,FGA_CORNER3_EWA,FGM_ABOVE_BREAK3_EWA,FGA_ABOVE_BREAK3_EWA,FG2M_EWA,FG2A_EWA,PTS_2PT_MR_EWA,AST_2PM_EWA,AST_3PM_EWA,UAST_2PM_EWA,UAST_3PM_EWA,OPP_FG2M_G_EWA,OPP_FG2A_G_EWA,OPP_FG2M_F_EWA,OPP_FG2A_F_EWA,OPP_FG2M_C_EWA,OPP_FG2A_C_EWA,MIN_opp_EWA,PTS_opp_EWA,FGM_opp_EWA,FGA_opp_EWA,FG3M_opp_EWA,FG3A_opp_EWA,FTM_opp_EWA,FTA_opp_EWA,OREB_opp_EWA,DREB_opp_EWA,REB_opp_EWA,AST_opp_EWA,TOV_opp_EWA,STL_opp_EWA,BLK_opp_EWA,PLUS_MINUS_opp_EWA,OFF_RATING_opp_EWA,DEF_RATING_opp_EWA,NET_RATING_opp_EWA,PACE_opp_EWA,PIE_opp_EWA,PCT_PTS_2PT_MR_opp_EWA,PCT_AST_2PM_opp_EWA,PCT_UAST_2PM_opp_EWA,PCT_AST_3PM_opp_EWA,PCT_UAST_3PM_opp_EWA,DEFENSIVE_BOX_OUTS_opp_EWA,CONTESTED_SHOTS_2PT_opp_EWA,CONTESTED_SHOTS_3PT_opp_EWA,deflections_opp_EWA,DIST_opp_EWA,TCHS_opp_EWA,PASS_opp_EWA,CFGM_opp_EWA,CFGA_opp_EWA,UFGM_opp_EWA,UFGA_opp_EWA,DFGM_opp_EWA,DFGA_opp_EWA,OPP_FG3M_C_opp_EWA,OPP_FG3A_C_opp_EWA,OPP_FG3M_F_opp_EWA,OPP_FG3A_F_opp_EWA,OPP_FG3M_G_opp_EWA,OPP_FG3A_G_opp_EWA,OPP_FTM_C_opp_EWA,OPP_FTM_F_opp_EWA,OPP_FTM_G_opp_EWA,OPP_FTA_C_opp_EWA,OPP_FTA_F_opp_EWA,OPP_FTA_G_opp_EWA,PTS_OFF_TOV_opp_EWA,PTS_2ND_CHANCE_opp_EWA,PTS_FB_opp_EWA,PTS_PAINT_opp_EWA,OPP_PTS_OFF_TOV_opp_EWA,OPP_PTS_2ND_CHANCE_opp_EWA,OPP_PTS_FB_opp_EWA,OPP_PTS_PAINT_opp_EWA,BLOCKS_opp_EWA,BLOCKED_ATT_opp_EWA,PF_opp_EWA,PFD_opp_EWA,FGM_RESTRICTED_opp_EWA,FGA_RESTRICTED_opp_EWA,FGM_PAINT_NON_RA_opp_EWA,FGA_PAINT_NON_RA_opp_EWA,FGM_MIDRANGE_opp_EWA,FGA_MIDRANGE_opp_EWA,FGM_CORNER3_opp_EWA,FGA_CORNER3_opp_EWA,FGM_ABOVE_BREAK3_opp_EWA,FGA_ABOVE_BREAK3_opp_EWA,IS_HOME_opp_EWA,FG2M_opp_EWA,FG2A_opp_EWA,PTS_2PT_MR_opp_EWA,AST_2PM_opp_EWA,AST_3PM_opp_EWA,UAST_2PM_opp_EWA,UAST_3PM_opp_EWA,OPP_FG2M_G_opp_EWA,OPP_FG2A_G_opp_EWA,OPP_FG2M_F_opp_EWA,OPP_FG2A_F_opp_EWA,OPP_FG2M_C_opp_EWA,OPP_FG2A_C_opp_EWA,POINT_DIFF_opp_EWA,RECORD_opp_EWA,TEAM_SCORE_opp_EWA
53,2016-17,1610612737,ATL,0021600014,2016-10-27,ATL vs. WAS,1,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,2016-17,1610612737,ATL,0021600026,2016-10-29,ATL @ PHI,1,104,114.000000,44.000000,88.000000,12.000000,26.000000,14.000000,18.000000,14.000000,38.000000,52.000000,28.000000,21.000000,13.000000,7.000000,15.000000,110.700000,97.100000,13.600000,102.500000,0.603000,0.175000,0.500000,0.500000,1.000000,0.000000,0.000000,54.000000,18.000000,21.000000,17.370000,432.000000,300.000000,21.000000,44.000000,23.000000,44.000000,12.000

In [23]:

def add_schedule_features(df):
    """Add features related to rest and schedule density"""
    df = df.copy()
    df = df.sort_values(['TEAM_ID', 'GAME_DATE'])

    # Calculate days between games for teams
    df['DAYS_REST'] = df.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
    
    # Fill NaN values for first games
    df['DAYS_REST'] = df['DAYS_REST'].fillna(5)
    

    # Back-to-back games
    df['B2B_tm'] = (df['DAYS_REST'] <= 1).astype(int)
    
    return df


team_final_df = add_schedule_features(df_team_with_ewa3)

team_final_df 

,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PTS_EWA,FGM_EWA,FGA_EWA,FG3M_EWA,FG3A_EWA,FTM_EWA,FTA_EWA,OREB_EWA,DREB_EWA,REB_EWA,AST_EWA,TOV_EWA,STL_EWA,BLK_EWA,PLUS_MINUS_EWA,OFF_RATING_EWA,DEF_RATING_EWA,NET_RATING_EWA,PACE_EWA,PIE_EWA,PCT_PTS_2PT_MR_EWA,PCT_AST_2PM_EWA,PCT_UAST_2PM_EWA,PCT_AST_3PM_EWA,PCT_UAST_3PM_EWA,DEFENSIVE_BOX_OUTS_EWA,CONTESTED_SHOTS_2PT_EWA,CONTESTED_SHOTS_3PT_EWA,deflections_EWA,DIST_EWA,TCHS_EWA,PASS_EWA,CFGM_EWA,CFGA_EWA,UFGM_EWA,UFGA_EWA,DFGM_EWA,DFGA_EWA,OPP_FG3M_C_EWA,OPP_FG3A_C_EWA,OPP_FG3M_F_EWA,OPP_FG3A_F_EWA,OPP_FG3M_G_EWA,OPP_FG3A_G_EWA,OPP_FTM_C_EWA,OPP_FTM_F_EWA,OPP_FTM_G_EWA,OPP_FTA_C_EWA,OPP_FTA_F_EWA,OPP_FTA_G_EWA,PTS_OFF_TOV_EWA,PTS_2ND_CHANCE_EWA,PTS_FB_EWA,PTS_PAINT_EWA,OPP_PTS_OFF_TOV_EWA,OPP_PTS_2ND_CHANCE_EWA,OPP_PTS_FB_EWA,OPP_PTS_PAINT_EWA,BLOCKS_EWA,BLOCKED_ATT_EWA,PF_EWA,PFD_EWA,FGM_RESTRICTED_EWA,FGA_RESTRICTED_EWA,FGM_PAINT_NON_RA_EWA,FGA_PAINT_NON_RA_EWA,FGM_MIDRANGE_EWA,FGA_MIDRANGE_EWA,FGM_CORNER3_EWA,FGA_CORNER3_EWA,FGM_ABOVE_BREAK3_EWA,FGA_ABOVE_BREAK3_EWA,FG2M_EWA,FG2A_EWA,PTS_2PT_MR_EWA,AST_2PM_EWA,AST_3PM_EWA,UAST_2PM_EWA,UAST_3PM_EWA,OPP_FG2M_G_EWA,OPP_FG2A_G_EWA,OPP_FG2M_F_EWA,OPP_FG2A_F_EWA,OPP_FG2M_C_EWA,OPP_FG2A_C_EWA,MIN_opp_EWA,PTS_opp_EWA,FGM_opp_EWA,FGA_opp_EWA,FG3M_opp_EWA,FG3A_opp_EWA,FTM_opp_EWA,FTA_opp_EWA,OREB_opp_EWA,DREB_opp_EWA,REB_opp_EWA,AST_opp_EWA,TOV_opp_EWA,STL_opp_EWA,BLK_opp_EWA,PLUS_MINUS_opp_EWA,OFF_RATING_opp_EWA,DEF_RATING_opp_EWA,NET_RATING_opp_EWA,PACE_opp_EWA,PIE_opp_EWA,PCT_PTS_2PT_MR_opp_EWA,PCT_AST_2PM_opp_EWA,PCT_UAST_2PM_opp_EWA,PCT_AST_3PM_opp_EWA,PCT_UAST_3PM_opp_EWA,DEFENSIVE_BOX_OUTS_opp_EWA,CONTESTED_SHOTS_2PT_opp_EWA,CONTESTED_SHOTS_3PT_opp_EWA,deflections_opp_EWA,DIST_opp_EWA,TCHS_opp_EWA,PASS_opp_EWA,CFGM_opp_EWA,CFGA_opp_EWA,UFGM_opp_EWA,UFGA_opp_EWA,DFGM_opp_EWA,DFGA_opp_EWA,OPP_FG3M_C_opp_EWA,OPP_FG3A_C_opp_EWA,OPP_FG3M_F_opp_EWA,OPP_FG3A_F_opp_EWA,OPP_FG3M_G_opp_EWA,OPP_FG3A_G_opp_EWA,OPP_FTM_C_opp_EWA,OPP_FTM_F_opp_EWA,OPP_FTM_G_opp_EWA,OPP_FTA_C_opp_EWA,OPP_FTA_F_opp_EWA,OPP_FTA_G_opp_EWA,PTS_OFF_TOV_opp_EWA,PTS_2ND_CHANCE_opp_EWA,PTS_FB_opp_EWA,PTS_PAINT_opp_EWA,OPP_PTS_OFF_TOV_opp_EWA,OPP_PTS_2ND_CHANCE_opp_EWA,OPP_PTS_FB_opp_EWA,OPP_PTS_PAINT_opp_EWA,BLOCKS_opp_EWA,BLOCKED_ATT_opp_EWA,PF_opp_EWA,PFD_opp_EWA,FGM_RESTRICTED_opp_EWA,FGA_RESTRICTED_opp_EWA,FGM_PAINT_NON_RA_opp_EWA,FGA_PAINT_NON_RA_opp_EWA,FGM_MIDRANGE_opp_EWA,FGA_MIDRANGE_opp_EWA,FGM_CORNER3_opp_EWA,FGA_CORNER3_opp_EWA,FGM_ABOVE_BREAK3_opp_EWA,FGA_ABOVE_BREAK3_opp_EWA,IS_HOME_opp_EWA,FG2M_opp_EWA,FG2A_opp_EWA,PTS_2PT_MR_opp_EWA,AST_2PM_opp_EWA,AST_3PM_opp_EWA,UAST_2PM_opp_EWA,UAST_3PM_opp_EWA,OPP_FG2M_G_opp_EWA,OPP_FG2A_G_opp_EWA,OPP_FG2M_F_opp_EWA,OPP_FG2A_F_opp_EWA,OPP_FG2M_C_opp_EWA,OPP_FG2A_C_opp_EWA,POINT_DIFF_opp_EWA,RECORD_opp_EWA,TEAM_SCORE_opp_EWA,DAYS_REST,B2B_tm
53,2016-17,1610612737,ATL,0021600014,2016-10-27,ATL vs. WAS,1,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0
101,2016-17,1610612737,ATL,0021600026,2016-10-29,ATL @ PHI,1,104,114.000000,44.000000,88.000000,12.000000,26.000000,14.000000,18.000000,14.000000,38.000000,52.000000,28.000000,21.000000,13.000000,7.000000,15.000000,110.700000,97.100000,13.600000,102.500000,0.603000,0.175000,0.500000,0.500000,1.000000,0.000000,0.000000,54.000000,18.000000,21.000000,17.370000,432.000000,300.000000,21.000000,44.000000,23.

In [ ]:
def get_todays_games():
    """
    Get today's NBA games using ScoreboardV2
    
    Returns:
    pd.DataFrame: DataFrame containing game information
    """
    try:
        # Get today's date in the format MM/DD/YYYY
        today = datetime.now().strftime('%Y/%m/%d')        
        # Get scoreboard for today
        games = scoreboardv2.ScoreboardV2(game_date='today').game_header.get_data_frame()
        

        
        # Extract relevant columns
        game_info = games[[
            'SEASON',
            'GAME_DATE_EST',
            'GAME_ID', 
            'GAME_STATUS_ID',  # 1: Scheduled, 2: In Progress, 3: Final
            'GAME_STATUS_TEXT',
            'HOME_TEAM_ID',
            'VISITOR_TEAM_ID',
        ]]
        
        return games
        
    except Exception as e:
        print(f"Error fetching games: {str(e)}")
        return None
    

get_todays_games()

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS,WNBA_COMMISSIONER_FLAG
0,2024-11-15T00:00:00,1,0022400009,2,1st Qtr,20241115/MIAIND,1610612754,1610612748,2024,1,1:53,None,FDSNIN,FDSNSU,Q1 1:53 -,Gainbridge Fieldhouse,0,0
1,2024-11-15T00:00:00,2,0022400010,2,1st Qtr,20241115/PHIORL,1610612753,1610612755,2024,1,2:19,None,FDSNFL,NBCSP,Q1 2:19 -,Kia Center,0,0
2,2024-11-15T00:00:00,3,0022400011,2,1st Qtr,20241115/DETTOR,1610612761,1610612765,2024,1,2:33,None,SN,FDSNDT,Q1 2:33 -,Scotiabank Arena,0,0
3,2024-11-15T00:00:00,4,0022400012,1,7:30 pm ET,20241115/WASATL,1610612737,1610612764,2024,0,,None,FDSNSE-ATL/WPCH,MNMT,Q0 -,State Farm Arena,0,0
4,2024-11-15T00:00:00,5,0022400013,1,7:30 pm ET,20241115/CHICLE,1610612739,1610612741,2024,0,,None,FDSNOH,CHSN,Q0 -,Rocket Mortgage FieldHouse,0,0
5,2024-11-15T00:00:00,6,0022400014,1,7:30 pm ET,20241115/BKNNYK,1610612752,1610612751,2024,0,,None,MSG,YES,Q0 -,Madison Square Garden,0,0
6,2024-11-15T00:00:00,7,0022400015,1,7:30 pm ET,20241115/LALSAS,1610612759,1610612747,2024,0,,ESPN,None,SPECSN,Q0 - ESPN,Frost Bank Center,0,0
7,2024-11-15T00:00:00,8,0022400016,1,8:00 pm ET,20241115/LACHOU,1610612745,1610612746,2024,0,,None,SCHN,FDSNSC,Q0 -,Toyota Center,0,0
8,2024-11-15T00:00:00,9,0022400017,1,8:00 pm ET,20241115/DENNOP,1610612740,1610612743,2024,0,,None,GCSEN,ALT,Q0 -,Smoothie King Center,0,0
9,2024-11-15T00:00:00,10,0022400018,1,8:00 pm ET,20241115/PHXOKC,1610612760,1610612756,2024,0,,None,FDSNOK,KTVK/KPHE,Q0 -,Paycom Center,0,0


In [ ]:
games = scoreboardv2.ScoreboardV2(game_date='today').game_header.get_data_frame()


In [3]:
games

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS,WNBA_COMMISSIONER_FLAG
0,2024-11-16T00:00:00,1,0022400229,3,Final,20241116/MILCHA,1610612766,1610612749,2024,4,,None,FDSNSE-CHA,FDSNWI,Q4 -,Spectrum Center,1,0
1,2024-11-16T00:00:00,2,0022400230,2,1st Qtr,20241116/TORBOS,1610612738,1610612761,2024,1,7:57,None,NBCSB,TSN,Q1 7:57 -,TD Garden,0,0
2,2024-11-16T00:00:00,3,0022400231,2,1st Qtr,20241116/LALNOP,1610612740,1610612747,2024,1,5:24,None,GCSEN,SPECSN,Q1 5:24 -,Smoothie King Center,0,0
3,2024-11-16T00:00:00,4,0022400232,1,8:30 pm ET,20241116/SASDAL,1610612742,1610612759,2024,0,,NBA TV,KFAA,KENS,Q0 - NBA TV,American Airlines Center,0,0
4,2024-11-16T00:00:00,5,0022400233,1,10:00 pm ET,20241116/UTASAC,1610612758,1610612762,2024,0,,None,NBCSCA,KJZZ,Q0 -,Golden 1 Center,0,0


In [45]:
def prepare_team_features(self, team_id: int, opponent_id: int) -> pd.DataFrame:
    """
    Prepare features for a team against specific opponent using latest aggregated data
    
    Parameters:
    team_id (int): ID of the team to prepare features for
    opponent_id (int): ID of the opposing team
    
    Returns:
    pd.DataFrame: Single row DataFrame with all features for prediction
    """
    try:
        with sqlite3.connect(self.db_path) as conn:
            # Get the latest EWA features for both teams
            query = """
            WITH RankedGames AS (
                SELECT *,
                    ROW_NUMBER() OVER (PARTITION BY TEAM_ID ORDER BY GAME_DATE DESC) as rn
                FROM df_team_with_ewa3  -- Your table with EWA features
                WHERE TEAM_ID IN (?, ?)
            )
            SELECT *
            FROM RankedGames
            WHERE rn = 1
            """
            
            latest_stats = pd.read_sql_query(query, conn, params=[team_id, opponent_id])
            
            if latest_stats.empty:
                logging.error(f"No stats found for teams {team_id} and {opponent_id}")
                return None
            
            # Separate team and opponent stats
            team_stats = latest_stats[latest_stats['TEAM_ID'] == team_id].iloc[0]
            opp_stats = latest_stats[latest_stats['TEAM_ID'] == opponent_id].iloc[0]
            
            # Get columns that end with _EWA (your exponentially weighted averages)
            ewa_cols = [col for col in latest_stats.columns if col.endswith('_EWA')]
            
            # Create feature dictionary
            features = {}
            
            # Add team EWA features
            for col in ewa_cols:
                features[f'{col}_tm'] = team_stats[col]
                features[f'{col}_opp'] = opp_stats[col]
            
            # Add schedule features
            features['DAYS_REST_tm'] = self.calculate_rest_days(team_id)
            features['DAYS_REST_opp'] = self.calculate_rest_days(opponent_id)
            
            # Add any other relevant features
            features['IS_HOME'] = 1  # This will be 1 for home team row, 0 for away team row
            
            # Convert to DataFrame
            features_df = pd.DataFrame([features])
            
            return features_df
            
    except Exception as e:
        logging.error(f"Error preparing features: {str(e)}")
        return None


2024-11-13


,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS,WNBA_COMMISSIONER_FLAG
0,2024-11-13T00:00:00,1,0022400217,1,7:00 pm ET,20241113/INDORL,1610612753,1610612754,2024,0,,None,FDSNFL,FDSNIN,Q0 -,Kia Center,0,0
1,2024-11-13T00:00:00,2,0022400218,1,7:30 pm ET,20241113/BOSBKN,1610612751,1610612738,2024,0,,None,YES,NBCSB,Q0 -,Barclays Center,0,0
2,2024-11-13T00:00:00,3,0022400219,1,7:30 pm ET,20241113/CHINYK,1610612752,1610612741,2024,0,,None,MSG,CHSN,Q0 -,Madison Square Garden,0,0
3,2024-11-13T00:00:00,4,0022400220,1,7:30 pm ET,20241113/CLEPHI,1610612755,1610612739,2024,0,,None,NBCSP,FDSNOH,Q0 -,Wells Fargo Center,0,0
4,2024-11-13T00:00:00,5,0022400221,1,7:30 pm ET,20241113/NOPOKC,1610612760,1610612740,2024,0,,ESPN,FDSNOK,None,Q0 - ESPN,Paycom Center,0,0
5,2024-11-13T00:00:00,6,0022400222,1,8:00 pm ET,20241113/LACHOU,1610612745,1610612746,2024,0,,None,SCHN,FDSNSC,Q0 -,Toyota Center,0,0
6,2024-11-13T00:00:00,7,0022400223,1,8:00 pm ET,20241113/DETMIL,1610612749,1610612765,2024,0,,None,FDSNWI,FDSNDT,Q0 -,Fiserv Forum,0,0
7,2024-11-13T00:00:00,8,0022400224,1,8:00 pm ET,20241113/WASSAS,1610612759,1610612764,2024,0,,None,FDSNSW,MNMT2,Q0 -,Frost Bank Center,0,0
8,2024-11-13T00:00:00,9,0022400225,1,10:00 pm ET,20241113/MEMLAL,1610612747,1610612763,2024,0,,ESPN,SPECSN,FDSNSE-MEM,Q0 - ESPN,Crypto.com Arena,0,0
9,2024-11-13T00:00:00,10,0022400226,1,10:00 pm ET,20241113/MINPOR,1610612757,1610612750,2024,0,,None,KATU.2,FDSNNO,Q0 -,Moda Center,0,0


In [30]:

def engineer_features(df):
    """
    Create engineered features for NBA points prediction
    
    Parameters:
    df (pd.DataFrame): DataFrame containing NBA game statistics with EWA metrics
    
    Returns:
    pd.DataFrame: Original dataframe with added engineered features
    """
    # Create copy to avoid modifying original
    df_new = df.copy()
    
    # 1. Offensive Matchup Features
    df_new['OFF_DEF_RATING_DELTA'] = df_new['OFF_RATING_EWA_tm'] - df_new['DEF_RATING_EWA_opp']
    
    # Calculate restricted area FG% safely
    df_new['FG_PCT_RESTRICTED_EWA_tm'] = df_new['FGM_RESTRICTED_EWA_tm'] / df_new['FGA_RESTRICTED_EWA_tm'].replace(0, 1)
    
    df_new['PAINT_SCORING_ADVANTAGE'] = (df_new['FG_PCT_RESTRICTED_EWA_tm'] * df_new['FGA_RESTRICTED_EWA_tm']) / \
                                      (df_new['OPP_FG2M_C_EWA_opp'] + df_new['OPP_FG2M_F_EWA_opp']).replace(0, 1)
    
    df_new['RELATIVE_PACE'] = df_new['PACE_EWA_tm'] / df_new['PACE_EWA_opp'].replace(0, 1)
    
    # 2. Style Matchup Features
    df_new['THREE_PT_RATIO_tm'] = df_new['FG3A_EWA_tm'] / df_new['FGA_EWA_tm'].replace(0, 1)
    df_new['PAINT_RATIO_tm'] = (df_new['FGA_RESTRICTED_EWA_tm'] + df_new['FGA_PAINT_NON_RA_EWA_tm']) / \
                               df_new['FGA_EWA_tm'].replace(0, 1)
    
    df_new['AST_SCORING_DIFF'] = (df_new['PCT_AST_2PM_EWA_tm'] + df_new['PCT_AST_3PM_EWA_tm']) - \
                                (df_new['PCT_AST_2PM_EWA_opp'] + df_new['PCT_AST_3PM_EWA_opp'])
    
    # 3. Efficiency Metrics
    df_new['EFF_SCORING'] = df_new['PTS_EWA_tm'] / \
                           (2 * (df_new['FGA_EWA_tm'] + 0.44 * df_new['FTA_EWA_tm'])).replace(0, 1)
    
    df_new['POSS_EFFICIENCY'] = df_new['PTS_EWA_tm'] / \
                               (df_new['FGA_EWA_tm'] + df_new['TOV_EWA_tm'] + 0.44 * df_new['FTA_EWA_tm']).replace(0, 1)
    
    # 4. Defensive Impact Features
    df_new['DEF_PRESSURE_SCORE'] = (df_new['CONTESTED_SHOTS_2PT_EWA_tm'] + 
                                   df_new['CONTESTED_SHOTS_3PT_EWA_tm'] + 
                                   df_new['deflections_EWA_tm']) / \
                                   df_new['TCHS_EWA_opp'].replace(0, 1)
    
    df_new['TRANSITION_DEF_EFF'] = df_new['OPP_PTS_FB_EWA_tm'] / df_new['PTS_FB_EWA_opp'].replace(0, 1)
    
    # 5. Game Context Features
    df_new['SCORING_TREND'] = df_new['PTS_EWA_tm'] - df_new['PTS_opp_EWA_tm']
    df_new['HOME_SCORING_IMPACT'] = df_new['IS_HOME_opp_EWA_tm'] * df_new['PTS_EWA_tm']
    
    # 6. Advanced Interaction Features
    df_new['OREB_SCORING_IMPACT'] = df_new['OREB_EWA_tm'] * df_new['PTS_2ND_CHANCE_EWA_tm'] / \
                                   df_new['FGA_EWA_tm'].replace(0, 1)
    
    df_new['BALL_MOVEMENT_EFF'] = (df_new['AST_EWA_tm'] * df_new['PASS_EWA_tm']) / \
                                 (df_new['TOV_EWA_tm'] + 1)
    
    # 7. Additional Efficiency Metrics
    df_new['ASSISTED_SCORING_EFF'] = (df_new['PCT_AST_2PM_EWA_tm'] * df_new['FG2M_EWA_tm'] + 
                                     df_new['PCT_AST_3PM_EWA_tm'] * df_new['FG3M_EWA_tm']) / \
                                    df_new['FGA_EWA_tm'].replace(0, 1)
    
    df_new['PAINT_DEF_EFF'] = df_new['OPP_PTS_PAINT_EWA_tm'] / \
                              (df_new['FGA_RESTRICTED_EWA_opp'] + df_new['FGA_PAINT_NON_RA_EWA_opp']).replace(0, 1)
    
    # Replace infinities with 0
    df_new = df_new.replace([np.inf, -np.inf], 0)
    
    # Fill any NaNs with 0
    df_new = df_new.fillna(0)
    
    return df_new



In [45]:

def add_schedule_features(df):
    """Add features related to rest and schedule density"""
    df = df.copy()
    
    # Calculate days between games for teams
    df['DAYS_REST'] = df.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
    
    # Fill NaN values for first games
    df['DAYS_REST'] = df['DAYS_REST'].fillna(5)
    
    # Games in last 7 days
    df['GAMES_LAST_7'] = df.groupby('TEAM_ID')['GAME_DATE'].transform(
        lambda x: x.rolling('7D').count() - 1
        )
    
    # Back-to-back games
    df['B2B_tm'] = (df['DAYS_REST'] <= 1).astype(int)
    
    return df


def add_matchup_history(df):
    """Add features based on historical matchups between teams"""
    df = df.copy()
    
    # Create unique matchup identifier
    df['MATCHUP_ID'] = df.apply(
        lambda x: '_'.join(sorted([str(x['TEAM_ID_tm']), str(x['TEAM_ID_opp'])])),
        axis=1
    )
    
    # Last matchup result and scoring
    df['LAST_MATCHUP_PTS_tm'] = df.groupby('MATCHUP_ID')['PTS_tm'].shift(1)
    df['LAST_MATCHUP_PTS_opp'] = df.groupby('MATCHUP_ID')['PTS_opp'].shift(1)
    
    # Average last 3 matchups
    df['AVG_LAST_3_PTS_tm'] = df.groupby('MATCHUP_ID')['PTS_tm'].transform(
        lambda x: x.rolling(3, min_periods=1).mean().shift(1)
    )
    df['AVG_LAST_3_PTS_opp'] = df.groupby('MATCHUP_ID')['PTS_opp'].transform(
        lambda x: x.rolling(3, min_periods=1).mean().shift(1)
    )
    
    return df

def add_season_context(df):
    """Add features related to season context"""
    df = df.copy()
    
    # Extract month for seasonality
    df['MONTH'] = pd.to_datetime(df['GAME_DATE']).dt.month
    
    # Create season progression feature (0-1 scale)
    df['SEASON_PROGRESS'] = df.groupby('SEASON')['GAME_DATE'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min())
    )
    
    # Season phase indicators
    df['EARLY_SEASON'] = (df['SEASON_PROGRESS'] < 0.25).astype(int)
    df['MID_SEASON'] = ((df['SEASON_PROGRESS'] >= 0.25) & 
                        (df['SEASON_PROGRESS'] < 0.75)).astype(int)
    df['LATE_SEASON'] = (df['SEASON_PROGRESS'] >= 0.75).astype(int)
    
    return df

# def enhanced_feature_engineering(df):
#     """
#     Complete feature engineering pipeline combining all enhancements
    
#     Parameters:
#     df (pd.DataFrame): Input DataFrame with base features
    
#     Returns:
#     pd.DataFrame: Enhanced DataFrame with all new features
#     """
#     df = (df
#         #   .pipe(engineer_features)       # Your existing feature engineering
#           .pipe(add_schedule_features)   # Add rest days and schedule density
#         #   .pipe(add_momentum_features)   # Add team momentum indicators
#         #   .pipe(add_matchup_history)     # Add historical matchup data
#         #   .pipe(add_season_context)      # Add season context
#          )
    
#     # Fill any remaining NaNs with 0
#     # df = df.fillna(0)
    
#     # Replace any infinities
#     df = df.replace([np.inf, -np.inf], 0)
    
#     return df

final_df = add_schedule_features(df_team_with_ewa3)

# Replace your final processing with this:
final_df = final_df.dropna()
final_df

ValueError: window must be an integer 0 or greater

In [46]:
df_team_with_ewa3

,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PTS_EWA,FGM_EWA,...,UAST_3PM_opp_EWA,OPP_FG2M_G_opp_EWA,OPP_FG2A_G_opp_EWA,OPP_FG2M_F_opp_EWA,OPP_FG2A_F_opp_EWA,OPP_FG2M_C_opp_EWA,OPP_FG2A_C_opp_EWA,POINT_DIFF_opp_EWA,RECORD_opp_EWA,TEAM_SCORE_opp_EWA
211,2016-17,1610612737,ATL,0021600014,2016-10-27,ATL vs. WAS,1,114,114.000000,44.000000,...,1.002000,13.400000,25.400000,18.000000,34.600000,5.600000,11.200000,-15.000000,0.000000,99.000000
214,2016-17,1610612737,ATL,0021600014,2016-10-27,ATL vs. WAS,1,114,114.000000,44.000000,...,1.002000,13.400000,25.400000,18.000000,34.600000,5.600000,11.200000,-15.000000,0.000000,99.000000
215,2016-17,1610612737,ATL,0021600014,2016-10-27,ATL vs. WAS,1,114,114.000000,44.000000,...,1.002000,13.400000,25.400000,18.000000,34.600000,5.600000,11.200000,-15.000000,0.000000,99.000000
305,2016-17,1610612737,ATL,0021600026,2016-10-29,ATL @ PHI,1,104,114.000000,44.000000,...,1.002000,13.400000,25.400000,18.000000,34.600000,5.600000,11.200000,-15.000000,0.000000,99.000000
369,2016-17,1610612737,ATL,0021600044,2016-10-31,ATL vs. SAC,1,106,113.333333,43.882353,...,1.039660,13.497561,25.491429,18.022857,34.364706,5.547170,11.112000,-15.871795,0.000000,97.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41302,2024-25,1610612766,CHA,0022400117,2024-10-30,CHA vs. TOR,1,138,106.705334,39.005931,...,1.707660,12.680624,22.403592,10.968559,20.094306,4.434718,8.202624,8.500009,0.673244,114.799250
41354,2024-25,1610612766,CHA,0022400132,2024-11-01,CHA vs. BOS,0,109,108.791645,39.593817,...,1.643220,12.062057,21.123387,10.341785,18.912288,4.267370,7.874519,7.807701,0.480889,115.902326
41394,2024-25,1610612766,CHA,0022400141,2024-11-02,CHA vs. BOS,0,103,108.805535,39.735358,...,1.694532,11.473664,19.916336,9.750825,17.799800,4.106338,7.559539,8.176537,0.629206,116.393094
41466,2024-25,1610612766,CHA,0022400162,2024-11-04,CHA @ MIN,0,93,108.418499,39.750925,...,1.781531,10.913973,18.778260,9.193635,16.752753,3.951382,7.257157,8.270048,0.735147,116.187452


In [9]:
conn = sqlite3.connect('../data/nba_stats.db')
tables = pd.read_sql("""select * from sqlite_master where type='table'""", conn)
team_aggregated_df = pd.read_sql("""select * from team_aggregated_stats""", conn)
conn.close()
tables

,type,name,tbl_name,rootpage,sql
0,table,traditional_stats,traditional_stats,2,CREATE TABLE traditional_stats (\n game_id ...
1,table,advanced_stats,advanced_stats,4,CREATE TABLE advanced_stats (\n game_id TEX...
2,table,scoring_stats,scoring_stats,6,CREATE TABLE scoring_stats (\n game_id TEXT...
3,table,defensive_stats,defensive_stats,8,CREATE TABLE defensive_stats (\n game_id TE...
4,table,hustle_stats,hustle_stats,10,"CREATE TABLE hustle_stats (\n game_id TEXT,..."
5,table,player_traditional_stats,player_traditional_stats,14,"CREATE TABLE ""player_traditional_stats"" (\n""GA..."
6,table,team_traditional_stats,team_traditional_stats,15,"CREATE TABLE ""team_traditional_stats"" (\n""GAME..."
7,table,player_advanced_stats,player_advanced_stats,16,"CREATE TABLE ""player_advanced_stats"" (\n""GAME_..."
8,table,team_advanced_stats,team_advanced_stats,20,"CREATE TABLE ""team_advanced_stats"" (\n""GAME_ID..."
9,table,player_scoring_stats,player_scoring_stats,21,"CREATE TABLE ""player_scoring_stats"" (\n""GAME_I..."


In [27]:
db_path = '../data/nba_stats.db'
season='2024-25'

from typing import List
def fetch_game_dates(season: str) -> List[str]:
    """
    Fetch all game dates for a given season
    
    Parameters:
    season (str): Season in YYYY-YY format
    
    Returns:
    List[str]: List of game dates in YYYY-MM-DD format
    """
    query = """
    SELECT DISTINCT GAME_DATE 
    FROM team_basic_stats 
    WHERE SEASON_YEAR = ? 
    ORDER BY GAME_DATE
    """
    
    with sqlite3.connect(db_path) as conn:
        dates = pd.read_sql_query(query, conn, params=[season])
        return dates['GAME_DATE'].tolist()
            
    

existing_dates_query = """
SELECT DISTINCT Date 
FROM defensive_stats_by_position 

"""

conn = sqlite3.connect(db_path)

existing_dates = set(pd.read_sql_query(
    existing_dates_query, 
    conn
)['Date'].tolist())

# Get all game dates

game_dates = fetch_game_dates(season)

print("game_dates:", game_dates)
print("existing_dates:", existing_dates)

# Process only missing dates
missing_dates = set(game_dates) - existing_dates
print("missing_dates:", missing_dates)

if not missing_dates:
    print(f"No missing dates for season {season}")

print(f"Found {len(missing_dates)} missing dates for season {season}")

# for date in missing_dates:
#     raw_data = self.get_all_defensive_stats(date, season)
#     processed_data = self.process_defensive_stats(raw_data)
    
#     if processed_data is not None:
#         processed_data.to_sql(
#             'defensive_stats_by_position',
#             conn,
#             if_exists='append',
#             index=False
#         )
#         print(f"Added defensive stats for {date}")

game_dates: ['2024-10-22', '2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26', '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02', '2024-11-03', '2024-11-04', '2024-11-06', '2024-11-07', '2024-11-08', '2024-11-09', '2024-11-10', '2024-11-11', '2024-11-12', '2024-11-13', '2024-11-14']
existing_dates: {'2015-12-06', '2018-11-01', '2016-02-26', '2017-10-21', '2022-03-24', '2022-01-16', '2016-05-26', '2023-12-21', '2017-12-22', '2021-03-15', '2017-04-08', '2015-04-30', '2018-04-20', '2019-11-22', '2016-10-25', '2020-08-18', '2016-03-10', '2021-03-11', '2018-01-10', '2024-03-30', '2019-05-09', '2017-02-09', '2019-12-16', '2022-11-21', '2016-05-03', '2019-11-27', '2022-04-07', '2019-02-03', '2022-05-24', '2015-04-19', '2020-01-17', '2015-05-15', '2015-02-05', '2017-05-11', '2016-04-26', '2020-01-30', '2023-03-29', '2022-01-17', '2014-11-09', '2022-12-11', '2017-05-14', '2021-12-13', '2021-05-20', '2021-01-10', '2019-04-09', '2018-03-27', '202

In [32]:
set(game_dates) - set(existing_dates)

{'2024-10-22',
 '2024-10-23',
 '2024-10-24',
 '2024-10-25',
 '2024-10-26',
 '2024-10-27',
 '2024-10-28',
 '2024-10-29',
 '2024-10-30',
 '2024-10-31',
 '2024-11-01',
 '2024-11-02',
 '2024-11-03',
 '2024-11-04',
 '2024-11-06',
 '2024-11-07',
 '2024-11-08',
 '2024-11-09',
 '2024-11-13'}

In [42]:
defensive_stats = pd.read_sql('select * from defensive_stats_by_position', conn)
defensive_stats = defensive_stats.drop(columns = ['GAME_DATE'])
defensive_stats = defensive_stats.rename(columns={'Date': 'GAME_DATE'})

defensive_stats.to_sql('defensive_stats_by_position', conn, if_exists='replace')

25700

In [46]:
defensive_stats = pd.read_sql('select * from defensive_stats_by_position', conn)
defensive_stats = defensive_stats.drop(columns=['index'])
defensive_stats

defensive_stats.to_sql('defensive_stats_by_position', conn, if_exists='replace', index=False)

26022

In [8]:
team_aggregated_df.loc[team_aggregated_df['TEAM_ABBREVIATION'] == 'ATL'].head(40)

,SEASON,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PTS_EWA,FGM_EWA,...,UAST_2PM_opp_EWA,UAST_3PM_opp_EWA,OPP_FG2M_G_opp_EWA,OPP_FG2A_G_opp_EWA,OPP_FG2M_F_opp_EWA,OPP_FG2A_F_opp_EWA,OPP_FG2M_C_opp_EWA,OPP_FG2A_C_opp_EWA,DAYS_REST,B2B_tm
0,2016-17,1610612737,ATL,0021600014,2016-10-27 00:00:00,ATL vs. WAS,1,114,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0
1,2016-17,1610612737,ATL,0021600026,2016-10-29 00:00:00,ATL @ PHI,1,104,114.000000,44.000000,...,14.008000,1.002000,13.400000,25.400000,18.000000,34.600000,5.600000,11.200000,2.0,0
2,2016-17,1610612737,ATL,0021600044,2016-10-31 00:00:00,ATL vs. SAC,1,106,113.333333,43.882353,...,13.785611,1.039660,13.497561,25.491429,18.022857,34.364706,5.547170,11.112000,2.0,0
3,2016-17,1610612737,ATL,0021600059,2016-11-02 00:00:00,ATL vs. LAL,0,116,112.844444,43.242215,...,13.575855,1.000428,13.278168,25.211918,17.564408,33.566782,5.564258,11.251520,2.0,0
4,2016-17,1610612737,ATL,0021600070,2016-11-04 00:00:00,ATL @ WAS,0,92,113.054815,43.110320,...,14.099974,1.000374,13.274355,25.085523,17.566442,33.474618,5.512776,11.105459,2.0,0
5,2016-17,1610612737,ATL,0021600084,2016-11-05 00:00:00,ATL vs. HOU,1,112,111.651160,42.456771,...,14.261309,1.000360,13.260972,25.252065,17.374074,33.270229,5.493426,11.101241,1.0,1
6,2016-17,1610612737,ATL,0021600100,2016-11-08 00:00:00,ATL @ CLE,1,110,111.674416,42.606373,...,14.135681,1.075818,13.336046,25.294804,17.604127,33.548451,5.504995,11.033191,3.0,0
7,2016-17,1610612737,ATL,0021600111,2016-11-09 00:00:00,ATL vs. CHI,1,115,111.562789,42.511881,...,14.183809,1.072995,13.368434,25.186529,17.386748,33.022071,5.448203,10.855864,1.0,1
8,2016-17,1610612737,ATL,0021600132,2016-11-12 00:00:00,ATL vs. PHI,1,117,111.791936,42.481770,...,14.229598,1.107901,13.311437,25.004442,17.387505,33.020773,5.370912,10.741629,3.0,0
9,2016-17,1610612737,ATL,0021600154,2016-11-15 00:00:00,ATL @ MIA,1,93,112.139140,42.806372,...,14.161287,1.141640,13.345026,25.004188,17.468219,33.113669,5.266349,10.495964,3.0,0


In [12]:
conn = sqlite3.connect('../data/nba_stats.db')
tbs = pd.read_sql('select * from team_basic_stats', conn)


team_ids = tbs[['TEAM_ID', 'TEAM_ABBREVIATION']].drop_duplicates().sort_values(['TEAM_ABBREVIATION'])
team_ids = team_ids.values.tolist()

team_id_dict = {team_id: abbreviation for team_id, abbreviation in team_ids}
team_id_dict

{1610612737: 'ATL',
 1610612751: 'BKN',
 1610612738: 'BOS',
 1610612766: 'CHA',
 1610612741: 'CHI',
 1610612739: 'CLE',
 1610612742: 'DAL',
 1610612743: 'DEN',
 1610612765: 'DET',
 1610612744: 'GSW',
 1610612745: 'HOU',
 1610612754: 'IND',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612763: 'MEM',
 1610612748: 'MIA',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612740: 'NOP',
 1610612752: 'NYK',
 1610612760: 'OKC',
 1610612753: 'ORL',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612759: 'SAS',
 1610612761: 'TOR',
 1610612762: 'UTA',
 1610612764: 'WAS'}

In [ ]:
        # # NBA Team ID to abbreviation mapping
        # NBA_TEAMS = {
        #     1610612737: 'ATL',  # Atlanta Hawks
        #     1610612738: 'BOS',  # Boston Celtics
        #     1610612751: 'BKN',  # Brooklyn Nets
        #     1610612766: 'CHA',  # Charlotte Hornets
        #     1610612741: 'CHI',  # Chicago Bulls
        #     1610612739: 'CLE',  # Cleveland Cavaliers
        #     1610612742: 'DAL',  # Dallas Mavericks
        #     1610612743: 'DEN',  # Denver Nuggets
        #     1610612765: 'DET',  # Detroit Pistons
        #     1610612744: 'GSW',  # Golden State Warriors
        #     1610612745: 'HOU',  # Houston Rockets
        #     1610612754: 'IND',  # Indiana Pacers
        #     1610612746: 'LAC',  # Los Angeles Clippers
        #     1610612747: 'LAL',  # Los Angeles Lakers
        #     1610612763: 'MEM',  # Memphis Grizzlies
        #     1610612748: 'MIA',  # Miami Heat
        #     1610612749: 'MIL',  # Milwaukee Bucks
        #     1610612750: 'MIN',  # Minnesota Timberwolves
        #     1610612740: 'NOP',  # New Orleans Pelicans
        #     1610612752: 'NYK',  # New York Knicks
        #     1610612760: 'OKC',  # Oklahoma City Thunder
        #     1610612753: 'ORL',  # Orlando Magic
        #     1610612755: 'PHI',  # Philadelphia 76ers
        #     1610612756: 'PHX',  # Phoenix Suns
        #     1610612757: 'POR',  # Portland Trail Blazers
        #     1610612758: 'SAC',  # Sacramento Kings
        #     1610612759: 'SAS',  # San Antonio Spurs
        #     1610612761: 'TOR',  # Toronto Raptors
        #     1610612762: 'UTA',  # Utah Jazz
        #     1610612764: 'WAS',  # Washington Wizards
        #     }
        